Строим запрос к нашему сервису

In [1]:
import requests

import urllib.request
import json 

In [2]:
# Подключение к Google drive

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
path2='/content/drive/MyDrive/Colab_Notebooks/ML_in_business/COLLAB_PROJ/My_project_ML_API_Collab/'

In [4]:
!pip install flask-ngrok

In [5]:
import pandas as pd
import numpy as np
from sklearn.metrics import roc_auc_score,roc_curve,scorer
import dill

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.metrics.scorer module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.metrics. Anything that cannot be imported from sklearn.metrics is now part of the private API.
  warnings.warn(message, FutureWarning)


In [7]:
X_test = pd.read_csv(path2+"X_test.csv")
y_test = pd.read_csv(path2+"y_test.csv")

In [9]:
test_one = X_test.iloc[:1]
print(test_one.values.tolist())

[[160.0, 1.0, 0.0, 0.0, 3.0, 256.0, 111.0, 60.0, 9.37, 11.5]]


In [7]:
# Пример данных
data = ("160.0", "1.0", "0.0", "0.0", "3.0", "256.0", "111.0", "60.0", "9.37", "11.5")
print(data)

# формируем запрос
def send_json(x):
    accountweeks, contractrenewal, dataplan, datausage, custservcalls, daymins, daycalls, monthlycharge, overagefee, roammins = x
    print(accountweeks)
    body = {'AccountWeeks': accountweeks, 
            'ContractRenewal': contractrenewal, 
            'DataPlan': dataplan, 
            'DataUsage': datausage,
            'CustServCalls': custservcalls, 
            'DayMins': daymins, 
            'DayCalls': daycalls, 
            'MonthlyCharge': monthlycharge, 
            'OverageFee': overagefee,
            'RoamMins': roammins} 
    #print(body)
    myurl = 'http://6400-35-221-47-149.ngrok.io/'+'predict'
    headers = {'content-type': 'application/json; charset=utf-8'}
    response = requests.post(myurl, json=body, headers=headers)
    return response.json()['predictions']

# обращение к серверу с запросом из одного набора (его построили руками выше - data)
if __name__ == '__main__':
    response = send_json(data)
    print('предсказание',response)

('160.0', '1.0', '0.0', '0.0', '3.0', '256.0', '111.0', '60.0', '9.37', '11.5')
160.0
{'AccountWeeks': '160.0', 'ContractRenewal': '1.0', 'DataPlan': '0.0', 'DataUsage': '0.0', 'CustServCalls': '3.0', 'DayMins': '256.0', 'DayCalls': '111.0', 'MonthlyCharge': '60.0', 'OverageFee': '9.37', 'RoamMins': '11.5'}
предсказание 0.27


Сделаем обработку массы запросов:

1. загрузим данные X_test

In [ ]:

path2='/content/gdrive/MyDrive/Colab_Notebooks/ML_in_business/COLLAB_PROJ/'

In [9]:
import pandas as pd
from sklearn.metrics import roc_auc_score,roc_curve,scorer
from urllib import request, parse

X_test = pd.read_csv(path2+"X_test.csv")
y_test = pd.read_csv(path2+"y_test.csv")

2. Передаем запрос одиночный из наших данных и посмотрим на ответ

In [11]:
X_test = X_test[['AccountWeeks', 'ContractRenewal', 'DataPlan', 'DataUsage', 'CustServCalls', 'DayMins', 'DayCalls', 'MonthlyCharge', 'OverageFee', 'RoamMins']].astype(str)
X_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 834 entries, 0 to 833
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   AccountWeeks     834 non-null    object
 1   ContractRenewal  834 non-null    object
 2   DataPlan         834 non-null    object
 3   DataUsage        834 non-null    object
 4   CustServCalls    834 non-null    object
 5   DayMins          834 non-null    object
 6   DayCalls         834 non-null    object
 7   MonthlyCharge    834 non-null    object
 8   OverageFee       834 non-null    object
 9   RoamMins         834 non-null    object
dtypes: object(10)
memory usage: 65.3+ KB


In [12]:
response = send_json(X_test[['AccountWeeks', 'ContractRenewal', 'DataPlan', 'DataUsage',
                             'CustServCalls', 'DayMins', 'DayCalls', 'MonthlyCharge', 'OverageFee', 'RoamMins']].iloc[0,:])

160
{'AccountWeeks': '160', 'ContractRenewal': '1', 'DataPlan': '0', 'DataUsage': '0.0', 'CustServCalls': '3', 'DayMins': '256.0', 'DayCalls': '111', 'MonthlyCharge': '60.0', 'OverageFee': '9.37', 'RoamMins': '11.5'}


In [13]:
response

0.27

3. Сделаем N запросов и оценим время

In [15]:
N = 50

In [16]:
%%time
predictions = X_test[['AccountWeeks', 'ContractRenewal', 'DataPlan', 'DataUsage',
                             'CustServCalls', 'DayMins', 'DayCalls', 'MonthlyCharge', 'OverageFee', 'RoamMins']].iloc[:N,:].apply(lambda x: send_json(x),1)

160
{'AccountWeeks': '160', 'ContractRenewal': '1', 'DataPlan': '0', 'DataUsage': '0.0', 'CustServCalls': '3', 'DayMins': '256.0', 'DayCalls': '111', 'MonthlyCharge': '60.0', 'OverageFee': '9.37', 'RoamMins': '11.5'}
132
{'AccountWeeks': '132', 'ContractRenewal': '1', 'DataPlan': '0', 'DataUsage': '0.0', 'CustServCalls': '3', 'DayMins': '240.1', 'DayCalls': '115', 'MonthlyCharge': '57.0', 'OverageFee': '9.02', 'RoamMins': '8.0'}
143
{'AccountWeeks': '143', 'ContractRenewal': '1', 'DataPlan': '0', 'DataUsage': '0.29', 'CustServCalls': '1', 'DayMins': '110.1', 'DayCalls': '113', 'MonthlyCharge': '36.9', 'OverageFee': '8.45', 'RoamMins': '9.2'}
84
{'AccountWeeks': '84', 'ContractRenewal': '1', 'DataPlan': '1', 'DataUsage': '2.03', 'CustServCalls': '4', 'DayMins': '207.5', 'DayCalls': '138', 'MonthlyCharge': '73.3', 'OverageFee': '10.05', 'RoamMins': '7.5'}
100
{'AccountWeeks': '100', 'ContractRenewal': '0', 'DataPlan': '0', 'DataUsage': '0.19', 'CustServCalls': '0', 'DayMins': '68.5', 'Da

In [17]:
predictions.values[:5]

array([0.27, 0.14, 0.04, 0.35, 0.29])

4. Посчитаем метрику roc-auc

In [18]:
y_test.head()

,Churn
0,0
1,0
2,0
3,0
4,0


In [19]:
from sklearn.metrics import roc_auc_score,roc_curve,scorer

In [20]:
roc_auc_score(y_score=predictions.values, y_true=y_test.values[:N])

0.9866666666666667

5. Анализ : в целом похоже